In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Nov 22 16:49:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import sys

print(sys.version)


3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]


In [ ]:
import torch # Import the torch module

print(torch.__version__) # Now you can access the __version__ attribute

2.5.1+cu121


**Install Required Libraries**

In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install scikit-learn
!pip install datasets

In [ ]:
data_path = '/content/isis_tweets.csv'


In [ ]:
pip install transformers datasets torch


In [ ]:
import zipfile
import os

# Path to the uploaded zip file
zip_path = "distilbert_model.zip"  # Replace with your uploaded file's name

# Extraction directory
extract_dir = "/content/distilbert_model"

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Model unzipped to: {extract_dir}")


Model unzipped to: /content/distilbert_model


**Import necessary libraries**

In [ ]:
pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=e90b00973ce588ca48fb52c47f56774379589d49a792067a86752263efa45a68
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect, DetectorFactory, LangDetectException

# Ensure reproducibility in langdetect
DetectorFactory.seed = 0

# Function to detect and filter English tweets
def filter_english_text(data):
    def detect_language(text):
        try:
            return detect(text)
        except LangDetectException:
            return 'unknown'  # Or return None or another value indicating failure

    # Apply language detection with error handling
    data['language'] = data['tweets'].apply(detect_language)

    # Filter for English tweets
    english_data = data[data['language'] == 'en']
    return english_data

# Filter the dataset
filtered_data = filter_english_text(data)

# Display the shape of the filtered dataset
print(f"Filtered dataset contains {filtered_data.shape[0]} rows.")

# Save filtered dataset for verification (optional)
filtered_data.to_csv("/content/filtered_reshuffle_ISIS.csv", index=False)

Filtered dataset contains 2885 rows.


In [ ]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader

# Load your dataset
file_path = "/content/filtered_reshuffle_ISIS.csv"  # Replace with your dataset's file path
data = pd.read_csv(file_path)

# Load your fine-tuned model and tokenizer
model_dir = "/content/distilbert_model/distilbert_model"  # Replace with your fine-tuned model's path
tokenizer = DistilBertTokenizer.from_pretrained(model_dir)
model = DistilBertForSequenceClassification.from_pretrained(model_dir)


In [ ]:
import pandas as pd
import torch
from transformers import DistilBertTokenizer

# Load your dataset
file_path = "/content/filtered_reshuffle_ISIS.csv"  # Replace with your dataset file path
data = pd.read_csv(file_path)

# Ensure all labels are integers
data["label"] = pd.to_numeric(data["label"], errors="coerce").fillna(0).astype(int)

# Load your tokenizer (from your fine-tuned model directory)
tokenizer_dir = "/content/distilbert_model/distilbert_model"  # Replace with your tokenizer directory
tokenizer = DistilBertTokenizer.from_pretrained(tokenizer_dir)

# Cleaning function for text
import re
def clean_text(text):
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Remove non-ASCII characters
    text = re.sub(r'[^\w\s]', '', text)        # Remove special characters except spaces
    return text

# Clean the tweets
data["tweets"] = data["tweets"].apply(clean_text)

# Tokenize the dataset
def tokenize_data(texts, labels):
    encodings = tokenizer(
        texts.tolist(),                  # Convert the pandas Series to a list
        truncation=True,
        padding="max_length",
        max_length=512,
        return_tensors="pt"              # Return PyTorch tensors
    )
    labels_tensor = torch.tensor(labels.tolist(), dtype=torch.long)  # Convert labels to PyTorch tensors
    return encodings, labels_tensor

# Tokenize the dataset
encodings, labels = tokenize_data(data["tweets"], data["label"])

# Save the tokenized dataset for future use
torch.save({
    "input_ids": encodings["input_ids"],
    "attention_mask": encodings["attention_mask"],
    "labels": labels
}, "tokenized_dataset.pt")

print("Tokenized dataset saved as 'tokenized_dataset.pt'")


Tokenized dataset saved as 'tokenized_dataset.pt'


In [ ]:
from torch.utils.data import Dataset
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch

# Load the previously saved tokenized dataset
tokenized_data = torch.load("tokenized_dataset.pt")
input_ids = tokenized_data["input_ids"]
attention_mask = tokenized_data["attention_mask"]
labels = tokenized_data["labels"]

# Define a custom dataset class for Hugging Face Trainer
class CustomDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx]
        }

# Create datasets
train_size = int(0.8 * len(labels))
val_size = len(labels) - train_size
train_input_ids, val_input_ids = input_ids[:train_size], input_ids[train_size:]
train_attention_mask, val_attention_mask = attention_mask[:train_size], attention_mask[train_size:]
train_labels, val_labels = labels[:train_size], labels[train_size:]

train_dataset = CustomDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = CustomDataset(val_input_ids, val_attention_mask, val_labels)

# Load your previously fine-tuned model and tokenizer
model_dir = "/content/distilbert_model/distilbert_model"  # Path to the fine-tuned model directory
model = DistilBertForSequenceClassification.from_pretrained(model_dir)
tokenizer = DistilBertTokenizer.from_pretrained(model_dir)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save results
    evaluation_strategy="epoch",    # Evaluate after each epoch
    save_strategy="epoch",          # Save model after each epoch
    logging_dir="./logs",           # Directory to save logs
    num_train_epochs=3,             # Number of epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,   # Batch size for evaluation
    save_total_limit=2,             # Limit the number of saved models
    load_best_model_at_end=True,    # Load the best model at the end
    logging_steps=10,               # Log every 10 steps
    weight_decay=0.01,              # Weight decay
    warmup_steps=500,               # Warmup steps for learning rate scheduler
)

# Define a function to compute evaluation metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model and print metrics
metrics = trainer.evaluate()
print("Evaluation Metrics:")
print(f"Accuracy: {metrics['eval_accuracy']:.4f}")
print(f"Precision: {metrics['eval_precision']:.4f}")
print(f"Recall: {metrics['eval_recall']:.4f}")
print(f"F1 Score: {metrics['eval_f1']:.4f}")

# Save the further fine-tuned model
model.save_pretrained("./distilbert_finetuned_v2")
tokenizer.save_pretrained("./distilbert_finetuned_v2")
print("Model and tokenizer saved to ./distilbert_finetuned_v2")


<ipython-input-52-aaedaedc5aa1>:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tokenized_data = torch.load("tokenized_dataset.pt")
/usr/local/lib/python3.10/dist-packages/

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000100,0.000064,1.000000,1.000000,1.000000,1.000000
2,0.000000,0.000004,1.000000,1.000000,1.000000,1.000000
3,0.000000,0.000002,1.000000,1.000000,1.000000,1.000000


Evaluation Metrics:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
Model and tokenizer saved to ./distilbert_finetuned_v2


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load the newly fine-tuned model
model = DistilBertForSequenceClassification.from_pretrained("./distilbert_finetuned_v2")
tokenizer = DistilBertTokenizer.from_pretrained("./distilbert_finetuned_v2")

# Example usage
text = "Sample text for classification"
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
outputs = model(**inputs)
predicted_label = outputs.logits.argmax().item()
print(f"Predicted Label: {predicted_label}")


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import tarfile

# Replace 'path_to_finetuned_model' with the location of your fine-tuned model
model_path = "/content/distilbert_model.tar.gz"

# Extract the model files from the archive to a temporary directory
with tarfile.open(model_path, 'r:gz') as archive:
    archive.extractall('/tmp/distilbert_model')

# Load the tokenizer from the extracted directory
tokenizer = DistilBertTokenizer.from_pretrained('/tmp/distilbert_model')

# Tokenize the training data
train_encodings = tokenizer(list(train_texts), truncation=True, padding="max_length", max_length=128)
test_encodings = tokenizer(list(test_texts), truncation=True, padding="max_length", max_length=128)

In [ ]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Convert labels to a list
train_labels = train_labels.tolist()
test_labels = test_labels.tolist()

# Create datasets
train_dataset = Dataset(train_encodings, train_labels)
test_dataset = Dataset(test_encodings, test_labels)


NameError: name 'train_labels' is not defined

In [ ]:
import pickle

# Save tokenized data
with open("train_dataset.pkl", "wb") as f:
    pickle.dump(train_dataset, f)

with open("test_dataset.pkl", "wb") as f:
    pickle.dump(test_dataset, f)


In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import tarfile

# Path to your fine-tuned model
model_path = "/content/distilbert_model.tar.gz"

# Extract the model to a temporary directory
with tarfile.open(model_path, 'r:gz') as archive:
    archive.extractall('/tmp/distilbert_model')  # Change path if needed

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('/tmp/distilbert_model')

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,No log,0.000036
2,No log,0.000014
3,No log,0.000011


TrainOutput(global_step=300, training_loss=0.006583695411682129, metrics={'train_runtime': 69.2279, 'train_samples_per_second': 69.293, 'train_steps_per_second': 4.334, 'total_flos': 158861527838208.0, 'train_loss': 0.006583695411682129, 'epoch': 3.0})

In [ ]:
import tarfile
import os

# Path to your uploaded tar.gz file
tar_path = "/content/updated_DISTILBERT_model.tar.gz"  # Replace with your file name

# Extract the tar.gz file
with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall()
    print("Model extracted.")


Model extracted.


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Path to the extracted model folder
model_path = "updated_DISTILBERT_model"  # Replace with the extracted folder name

# Load the fine-tuned model and tokenizer
# Specify local_files_only=True to load from local directory
model = AutoModelForSequenceClassification.from_pretrained("/content/updated_DISTILBERT_model.tar.gz")
tokenizer = AutoTokenizer.from_pretrained("/content/updated_DISTILBERT_model.tar.gz")


print("Model and tokenizer loaded.")

OSError: Incorrect path_or_model_id: '/content/updated_DISTILBERT_model.tar.gz'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:
import shutil
import tarfile
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Path to the .tar.gz archive
model = "/content/updated_DISTILBERT_model.tar.gz"

# Extract the archive to a temporary directory
extract_dir = "/tmp/"

In [ ]:
import shutil
import tarfile
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Path to the .tar.gz archive
model_path = "/content/updated_DISTILBERT_model.tar.gz"

# Extract the archive to a temporary directory
extract_dir = "/tmp/updated_DISTILBERT_model"  # Create a subdirectory for the model

# Extract the model files to the specified directory
with tarfile.open(model_path, 'r:gz') as archive:
    archive.extractall(extract_dir)

# Load the model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(extract_dir, local_files_only=True) # Load the model from the subdirectory
tokenizer = AutoTokenizer.from_pretrained(extract_dir, local_files_only=True) # Load the tokenizer from the subdirectory


def classify_text(text, model, tokenizer):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Perform inference
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract logits and determine the class
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=-1).item()

    # Map the predicted class to labels (adjust based on your training labels)
    labels = {0: "No Peer Pressure (Positive)", 1: "Negative Peer Pressure"}
    return labels.get(predicted_class, "Unknown")

# Test the model
test_sentence = "I think you should skip school; everyone is doing it!"
classification = classify_text(test_sentence, model, tokenizer)
print(f"Input: {test_sentence}\nClassification: {classification}")

OSError: Can't load tokenizer for '/tmp/updated_DISTILBERT_model'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/tmp/updated_DISTILBERT_model' is the correct path to a directory containing all relevant files for a DistilBertTokenizerFast tokenizer.

In [ ]:
import shutil

# Create a tar.gz archive of the saved model directory
shutil.make_archive(save_directory, 'gztar', save_directory)

print(f"Compressed model saved as {save_directory}.tar.gz")


NameError: name 'save_directory' is not defined

# **Initialize Tokenizer**

In [ ]:
# Step 2: Initialize the DistilBert tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Tokenize the dataset
tokenized_df = dataset.map(tokenize_function, batched=True)

# Display the tokenized dataset
tokenized_df.head()


Map:   0%|          | 0/114504 [00:00<?, ? examples/s]

AttributeError: 'Dataset' object has no attribute 'head'

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import re

# Step 1: Load your already fine-tuned model
model_path = "/content/extraction"  # Local path to your saved model
tokenizer = DistilBertTokenizer.from_pretrained(model_path)
model = DistilBertForSequenceClassification.from_pretrained(model_path)

# Step 2: Load your new dataset
df = pd.read_csv("/content/reshuffle_ISIS.csv")

# ----- Data Preprocessing -----
def preprocess_text(text):
    """Preprocesses text by removing URLs, mentions, and special characters."""
    text = str(text)  # Ensure the input is a string
    text = re.sub(r"http\S+", "", text)  # Remove URLs
    text = re.sub(r"@\S+", "", text)  # Remove mentions
    text = re.sub(r"[^a-zA-Z0-9 ]", "", text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading/trailing whitespace
    return text

df['tweets'] = df['tweets'].apply(preprocess_text)

# Ensure no empty or invalid entries remain
df = df[df['tweets'].str.strip() != ""]  # Remove empty rows after preprocessing

# ----- End of Data Preprocessing -----

# Step 3: Split the dataset into train and test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["tweets"], df["label"], test_size=0.2, random_state=42
)

# Convert splits into dataframes
train_df = pd.DataFrame({"text": train_texts, "label": train_labels})
test_df = pd.DataFrame({"text": test_texts, "label": test_labels})

# Save to CSV for compatibility with `datasets`
train_df.to_csv("isis_train.csv", index=False)
test_df.to_csv("isis_test.csv", index=False)

# Step 4: Load the CSVs as datasets
data_files = {"train": "isis_train.csv", "test": "isis_test.csv"}
raw_datasets = load_dataset('csv', data_files=data_files)

# Step 5: Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# Prepare the dataset for PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(["text"])  # Ensure "text" matches your column name
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# Ensure labels are in torch.long (int64)
tokenized_datasets = tokenized_datasets.map(
    lambda examples: {'tweets': examples['labels'].type(torch.long)},
    batched=True
)

# Step 6: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    evaluation_strategy="epoch",  # Evaluate at the end of every epoch
    save_strategy="epoch",        # Save at the end of every epoch
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Load the best model based on the evaluation metric
    metric_for_best_model="accuracy",  # Replace with your desired metric
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    push_to_hub=False,
)

# Step 7: Fine-Tune the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)

trainer.train()

# Step 8: Save the Model
model.save_pretrained("./ISIS_BERT_fine_tuned_model")
tokenizer.save_pretrained("./ISIS_BERT_fine_tuned_model")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2683 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Map:   0%|          | 0/2683 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-76-a96ca2f8b48a>:86: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Float'

# **Step 3: Split the data into training and validation sets**

In [ ]:
# Step 3: Split the dataset into training and validation sets
train_size = int(0.8 * len(tokenized_df))  # 80% for training
train_dataset = tokenized_df.select(range(train_size))
eval_dataset = tokenized_df.select(range(train_size, len(tokenized_df)))

# Display the number of samples in each dataset
len(train_dataset), len(eval_dataset)


NameError: name 'tokenized_df' is not defined

# **Step 4: Initialize the model and set up the training arguments**

In [ ]:
# Step 4: Initialize the DistilBert model
from transformers import DistilBertForSequenceClassification, TrainingArguments Trauber

# Load the model with number of labels
num_labels = len(df['labels'].unique())  # Number of unique labels (2 in this case)

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# Set up the training arguments
training_args = TrainingArguments(
    output_dir='./results',
    run_name="distil_text_multi_class",  # Unique run name
    num_train_epochs=3,                 # Number of epochs
    per_device_train_batch_size=16,     # Batch size per device
    per_device_eval_batch_size=64,      # Batch size during evaluation
    warmup_steps=500,                   # Warmup steps for learning rate scheduler
    weight_decay=0.01,                  # Weight decay
    logging_dir='./logs',               # Logging directory
    logging_steps=10,                   # Log every 10 steps
    evaluation_strategy="epoch",        # Evaluate every epoch
    save_strategy="epoch",              # Save checkpoint every epoch
    load_best_model_at_end=True,        # Load the best model when training is finished
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# **Step 5: Define the Trainer and start the training process**

In [ ]:
# Step 5: Define the Trainer
from transformers import Trainer

trainer = Trainer(
    model=model,                         # The model to be trained
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=eval_dataset,           # Validation dataset
)

# Start training the model
trainer.train()


ValueError: Target size (torch.Size([16])) must be the same as input size (torch.Size([16, 3]))

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
from transformers import DistilBertTokenizer

# Load the dataset
data_path = '/content/FINAL_FINAL_RESHUFFLE.csv'  # Path to the new dataset
df = pd.read_csv(data_path)

# Ensure 'labels' column is consistent with 0 for negative and 1 for positive
# You already have the labels in the correct format (0 and 1), so no need to remap it unless needed
# If final_label exists with text 'negative' and 'positive', use the map function
# df['labels'] = df['final_label'].map({'negative': 0, 'positive': 1})  # If needed, use this instead

# Convert the pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['Text'], padding="max_length", truncation=True)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(lambda x: tokenize_function(x), batched=True)

# Split the dataset into training (80%) and evaluation (20%) sets
train_size = int(0.8 * len(tokenized_datasets))
train_dataset = tokenized_datasets.select(range(train_size))
eval_dataset = tokenized_datasets.select(range(train_size, len(tokenized_datasets)))

# Load the DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',                # Directory to save results
    run_name="distilbert_text_training",    # Run name for logging
    num_train_epochs=3,                    # Number of epochs
    per_device_train_batch_size=16,        # Batch size for training
    per_device_eval_batch_size=64,         # Batch size for evaluation
    warmup_steps=500,                      # Warmup steps for learning rate scheduler
    weight_decay=0.01,                     # Weight decay
    logging_dir='./logs',                  # Logging directory
    logging_steps=10,                      # Log every 10 steps
    evaluation_strategy="epoch",           # Evaluate after every epoch
    save_strategy="epoch",                 # Save model every epoch
    load_best_model_at_end=True,           # Load the best model when training ends
)

# Define the Trainer
trainer = Trainer(
    model=model,                          # The model to be trained
    args=training_args,                   # Training arguments
    train_dataset=train_dataset,          # Training dataset
    eval_dataset=eval_dataset,            # Evaluation dataset
)

# Start training the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.301700,0.274151
2,0.190600,0.315899
3,0.006400,0.502091


TrainOutput(global_step=21819, training_loss=0.20344769230197032, metrics={'train_runtime': 17564.8778, 'train_samples_per_second': 19.874, 'train_steps_per_second': 1.242, 'total_flos': 4.624211692503245e+16, 'train_loss': 0.20344769230197032, 'epoch': 3.0})

In [ ]:
!pip install datasets transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


**Save the Model**

In [ ]:
# Save the trained model and tokenizer
model.save_pretrained('./distilbert-model')
tokenizer.save_pretrained('./distilbert-model')

NameError: name 'model' is not defined

**Resume Training from Checkpoints**

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Reload model and tokenizer from checkpoint
model = DistilBertForSequenceClassification.from_pretrained('/content/drive/MyDrive/final_distilbert_model')
tokenizer = DistilBertTokenizer.from_pretrained('/content/drive/MyDrive/final_distilbert_model')

# Initialize the Trainer again
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# Continue training
trainer.train(resume_from_checkpoint=True)


In [ ]:
import tarfile

# Extract the tar.gz file
tar_path = "/content/distilbert_model.tar.gz"  # Path to your tar.gz file
extract_path = "/content/extraction"  # Target folder for extraction

with tarfile.open(tar_path, "r:gz") as tar:
    tar.extractall(extract_path)


# **Test the model in Colab**

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch.nn.functional as F

# Load the saved model and tokenizer
model = DistilBertForSequenceClassification.from_pretrained('/content/extraction')
tokenizer = DistilBertTokenizer.from_pretrained('/content/extraction')

# Example text for prediction
text = "That dress looks good except not on you cause you look fat like a truck"

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Get the model predictions
outputs = model(**inputs)
logits = outputs.logits

# Calculate confidence scores using softmax
probs = F.softmax(logits, dim=-1)
confidence_negative = probs[0][0].item()  # Confidence for Negative Peer Pressure
confidence_positive = probs[0][1].item()  # Confidence for No Peer Pressure

# Get the predicted label
predictions = logits.argmax(dim=-1)

# Print the prediction and confidence
if predictions.item() == 0:
    print(f"Predicted label: Negative Peer Pressure with confidence {confidence_negative:.2f}")
else:
    print(f"Predicted label: No Peer Pressure (Positive) with confidence {confidence_positive:.2f}")


Predicted label: No Peer Pressure (Positive) with confidence 0.59


In [ ]:
# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Perform inference
outputs = model(**inputs)
logits = outputs.logits
confidence = logits.softmax(dim=-1)  # Get probabilities for each class

# Get predictions
predictions = logits.argmax(dim=-1)

# Interpret predictions with confidence scores
if predictions.item() == 0:
    print(f"Predicted label: Negative Peer Pressure with confidence {confidence[0][0].item():.2f}")
else:
    print(f"Predicted label: No Peer Pressure (Positive) with confidence {confidence[0][1].item():.2f}")


Predicted label: Negative Peer Pressure with confidence 0.68


**Steps to Save and Transfer the Model to AWS SageMaker**

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Save the trained model and tokenizer to Google Drive or a local directory
model.save_pretrained('/content/drive/MyDrive/distilbert-model')
tokenizer.save_pretrained('/content/drive/MyDrive/distilbert-model')


('/content/drive/MyDrive/distilbert-model/tokenizer_config.json',
 '/content/drive/MyDrive/distilbert-model/special_tokens_map.json',
 '/content/drive/MyDrive/distilbert-model/vocab.txt',
 '/content/drive/MyDrive/distilbert-model/added_tokens.json')

In [ ]:
from google.colab import drive
import shutil
# Move the saved model directory to your Google Drive
drive_path = '/MyDrive/DistilBERT'  # Path in Google Drive where you want to save it

# Define model_save_path, assuming it was saved to './distilbert-model'
model_save_path = './distilbert-model'

shutil.move(model_save_path, drive_path)

print(f"Model and tokenizer are saved to {drive_path}")

Model and tokenizer are saved to /MyDrive/DistilBERT


In [ ]:
from transformers import DistilBertForSequenceClassification

# Save the model
model.save_pretrained("./distilbert_model")
tokenizer.save_pretrained("./distilbert_model")

# Create a tar.gz file for SageMaker
import shutil

# Make the .tar.gz file
shutil.make_archive("/content/distilbert_model", 'gztar', "./distilbert_model")


'/content/distilbert_model.tar.gz'

# **Relabel and Reshuffle**



In [ ]:
import pandas as pd

# Load the CSV dataset
data_path = '/content/isis_tweets.csv'  # The path to your uploaded dataset
df = pd.read_csv(data_path)

# Reshuffle the dataset
df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save the reshuffled dataset
shuffled_data_path = '/content/reshuffle_ISIS.csv'
df_shuffled.to_csv(shuffled_data_path, index=False)

# Output the first few rows of the reshuffled dataset
df_shuffled.head(), shuffled_data_path

(                                              tweets  label
 0  Make no mistake; this is not in #Italy nor in ...      0
 1  RT @MaghrebiQM: KSA &amp; Turkey condemn Assad...      0
 2  #IslamicState accepts responsibility for today...      0
 3  Hot:\nHeavy losses to #Assad army as BMP vbied...      0
 4  @Somaliyyah @ProJ32 cant xplain everything cau...      0,
 '/content/reshuffle_ISIS.csv')

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
